## Initial Cleaning


In [1]:
import pandas as pd
pd.options.display.max_columns = 99
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

Let's count the number of missing datapoints in each column. 

In [2]:
missing_values_list = []
for chunk in chunk_iter:
    missing_values_list.append(chunk.isnull().sum())
    
combined_missing_counts = pd.concat(missing_values_list)
unique_combined_missing_counts = combined_missing_counts.groupby(combined_missing_counts.index).sum()
unique_combined_missing_counts.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

Let's investigate the memory footprint of each column, and for the entire dataset in MB.

In [3]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
counter = 0
series_memory_footprint = pd.Series(dtype='float64')
for chunk in chunk_iter:
    if counter == 0:
        series_memory_footprint = chunk.memory_usage(deep=True) / 2**20
    else:
        series_memory_footprint += chunk.memory_usage(deep=True) / 2**20
    counter += 1

series_memory_footprint = series_memory_footprint.drop('Index')
series_memory_footprint

company_permalink         3.869808
company_name              3.424955
company_category_code     3.262619
company_country_code      3.025223
company_state_code        2.962161
company_region            3.253503
company_city              3.343473
investor_permalink        4.749821
investor_name             3.734270
investor_category_code    0.593590
investor_country_code     2.524654
investor_state_code       2.361876
investor_region           3.238946
investor_city             2.751430
funding_round_type        3.252704
funded_at                 3.378091
funded_month              3.226837
funded_quarter            3.226837
funded_year               0.403366
raised_amount_usd         0.403366
dtype: float64

In [4]:
total_memory_footprint = sum(series_memory_footprint)
total_memory_footprint

56.98753070831299

Not all columns are useful for our analysis. We can drop the `investor_category_code` columns as nearly all of the rows have missing data. The `company_permalink` and `investor_permalink` columns have URLs which are not useful, so they can also be dropped. 

In [5]:
cols = chunk.columns.drop(['investor_permalink', 'company_permalink', 'investor_category_code'])

Let's look at what datatypes each column is.

In [6]:
col_types = {}
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=cols)

for chunk in chunk_iter:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

## Creating SQLite Database


In [7]:
chunk

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52865,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,Zohar Gilon,NaN,NaN,unknown,NaN,series-a,2012-08-08,2012-08,2012-Q3,2012,3800000.0
52866,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,Zohar Gilon,NaN,NaN,unknown,NaN,series-c+,2013-04-08,2013-04,2013-Q2,2013,10300000.0
52867,SiteBrains,software,USA,CA,SF Bay,San Francisco,zohar israel,NaN,NaN,unknown,NaN,angel,2010-08-01,2010-08,2010-Q3,2010,350000.0
52868,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,Zorba Lieberman,NaN,NaN,unknown,NaN,series-a,2013-07-11,2013-07,2013-Q3,2013,8400000.0


Finally, let's import the chunks inte SQLite.

In [8]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

for chunk in chunk_iter:
    chunk.to_sql("investments", conn, if_exists='append', index=False)


Let's now query our new database with some SQL commands.

In [9]:
q = '''SELECT distinct company_category_code, count(*) counts
FROM investments
GROUP BY company_category_code
ORDER BY counts DESC
'''
company_category = pd.read_sql(q, conn)
print(company_category)

   company_category_code  counts
0               software   14486
1                    web   10030
2                biotech    9902
3             enterprise    8978
4                 mobile    8134
5            advertising    6400
6              ecommerce    4336
7              cleantech    3896
8            games_video    3786
9              analytics    3726
10              hardware    3074
11               medical    2630
12         semiconductor    2584
13       network_hosting    2150
14              security    1992
15               finance    1862
16                social    1840
17             education    1566
18                health    1340
19      public_relations    1318
20                  None    1286
21                search    1264
22                 other     962
23             messaging     904
24               fashion     736
25                  news     726
26                travel     674
27           hospitality     662
28         manufacturing     620
29        

In [10]:
q = '''SELECT company_name, raised_amount_usd, funded_year
FROM investments
WHERE raised_amount_usd > 500000000
GROUP BY company_name
ORDER BY raised_amount_usd DESC
'''
large_investments = pd.read_sql(q, conn)
print(large_investments)

             company_name  raised_amount_usd  funded_year
0               sigmacare       2.600000e+09         2013
1                Facebook       1.500000e+09         2011
2               Clearwire       1.500000e+09         2009
3          Wave Broadband       1.050000e+09         2012
4                     AOL       1.000000e+09         2005
5                 Groupon       9.500000e+08         2011
6  University of Maryland       7.500000e+08         2013
7            Vivint, Inc.       5.650000e+08         2010
8                Solyndra       5.350000e+08         2009
9       Fisker Automotive       5.290000e+08         2009
